In [1]:
# Import necessary libraries
import pandas as pd

In [2]:
class TrafficStressCalculator:
    def segment_lts(self, segment_type, bike_lane_width, parking_lane_width, speed_85, num_lanes, traffic_flow_direction, adt, has_centerline):

        """
        Calculate the LTS for a segment based on segment type and traffic characteristics.

        :param segment_type: Type of segment ('separated_path', 'mixed_traffic', 'bike_lane_no_parking', 'bike_lane_with_parking').
        :param bike_lane_width: Width of bike lane (meters).
        :param parking_lane_width: Width of parking lane adjacent to bike lane.
        :param speed_85: 85th percentile speed of the road (km/h).
        :param num_lanes: Number of traffic lanes.
        :param traffic_flow_direction: Direction of circulation for the segment.
        :param adt: Average daily traffic volume (ADT).
        :param has_centerline: Boolean, if centerline exists (for mixed traffic).
        :return: LTS level for the segment (1 to 4).
        """

        # Separated Path Segment
        if segment_type == 'separated_path':
            return 1  # if not busy_driveway else 2


        # Mixed Traffic Segment
        elif segment_type == 'mixed_traffic':

            # Case 1: Narrow one-way street
            if num_lanes == 1 and traffic_flow_direction == 1:
                if adt <= 600:
                    return 1 if speed_85 <= 46 else 2 if speed_85 <= 62 else 3
                elif adt <= 1_000:
                    return 2 if speed_85 <= 54 else 3 if speed_85 <= 70 else 4
                else:  # For adt > 1,000
                    return 2 if speed_85 <= 38 else 3 if speed_85 <= 62 else 4

            # Case 2: Unlaned two-way street (no centerline)
            elif num_lanes == 1 and not has_centerline:
                if adt <= 750:
                    return 1 if speed_85 <= 46 else 2 if speed_85 <= 62 else 3
                elif adt <= 1_500:
                    return 1 if speed_85 <= 46 else 2 if speed_85 <= 54 else 3 \
                    if speed_85 <= 70 else 4
                elif adt <= 3_000:
                    return 2 if speed_85 <= 54 else 3 if speed_85 <= 62 else 4
                else:  # For adt > 3,000
                    return 2 if speed_85 <= 46 else 3 if speed_85 <= 62 else 4

            # Case 3: Two-way street with one through lane per direction and a centerline
            elif num_lanes == 1 and has_centerline:
                if adt <= 1_000:
                    return 1 if speed_85 <= 46 else 2 if speed_85 <= 62 else 3
                elif adt <= 1_500:
                    return 2 if speed_85 <= 54 else 3 if speed_85 <= 70 else 4
                else:  # For adt > 1,500
                    return 2 if speed_85 <= 38 else 3 if speed_85 <= 62 else 4

            # Case 4: Two through lanes per direction
            elif num_lanes == 2:
                if adt <= 8_000:
                    return 3 if speed_85 <= 62 else 4
                else:  # For adt > 8,000
                    return 3 if speed_85 <= 46 else 4

            # Case 5: Three or more through lanes per direction
            else:
                return 3 if speed_85 <= 46 else 4


        # Bike Lane without Adjacent Parking
        elif segment_type == 'bike_lane_no_parking':

            # Case 1: One through lane per direction or unlaned
            if num_lanes == 1:
                if bike_lane_width >= 1.8:
                    return 1 if speed_85 <= 54 else 2 if speed_85 <= 62 else 3
                else: # for bike_lane_width < 1.8
                    return 2 if speed_85 <= 62 else 3 if speed_85 <= 78 else 4

            # Case 2: Two through lanes per direction
            elif num_lanes == 2:
                if bike_lane_width >= 1.8:
                    return 2 if speed_85 <= 62 else 3
                else: # for bike_lane_width < 1.8
                    return 2 if speed_85 <= 62 else 3 if speed_85 <= 70 else 4

            # Case 3: Three or more through lanes per direction
            else:
                if bike_lane_width: # any bike lane width
                    return 3 if speed_85 <= 62 else 4


        # Bike Lane with Adjacent Parking
        elif segment_type == 'bike_lane_with_parking':

            # Case 1: One through lane per direction or unlaned
            if num_lanes == 1:
                if bike_lane_width + parking_lane_width >= 4.45:
                    return 1 if speed_85 <= 46 else 2 if speed_85 <= 62 else 3
                else: # for reach (bike_lane_width + parking_lane_width) < 4.45
                    return 2 if speed_85 <= 54 else 3

            # Case 2: Two through lanes per direction
            elif num_lanes == 2:
                if bike_lane_width + parking_lane_width >= 4.45:
                    return 2 if speed_85 <= 46 else 3
                else: # for reach (bike_lane_width + parking_lane_width) < 4.45
                    return 3

            # Case 3: All other multilane configurations
            else:
                return 3
        else:
            # Default to highest stress level if no condition above matches
            return 4

In [3]:
# Load data
file_path = '/content/drive/MyDrive/Colab/LTS/sample_data.xlsx'
df = pd.read_excel(file_path, sheet_name='lts')
df.head()

,segment_type,bike_lane_width,parking_lane_width,speed_85,num_lanes,traffic_flow_direction,adt,has_centerline
0,separated_path,1.8,0.0,30,1,2,500,False
1,separated_path,1.5,0.0,50,2,1,1500,True
2,mixed_traffic,NaN,0.0,40,1,1,500,False
3,mixed_traffic,NaN,0.0,50,2,2,2000,True
4,bike_lane_no_parking,1.5,0.0,35,1,2,500,False


In [4]:
# Calculate segment LTS
calc = TrafficStressCalculator()

df['segment_LTS'] = df.apply(lambda row: calc.segment_lts(
    row['segment_type'], row['bike_lane_width'], row['parking_lane_width'], row['speed_85'],
    row['num_lanes'], row['traffic_flow_direction'], row['adt'], row['has_centerline']
), axis=1)

df['segment_LTS'] = df['segment_LTS'].fillna(0).apply(lambda x: int(round(x)))

df

,segment_type,bike_lane_width,parking_lane_width,speed_85,num_lanes,traffic_flow_direction,adt,has_centerline,segment_LTS
0,separated_path,1.8,0.0,30,1,2,500,False,1
1,separated_path,1.5,0.0,50,2,1,1500,True,1
2,mixed_traffic,NaN,0.0,40,1,1,500,False,1
3,mixed_traffic,NaN,0.0,50,2,2,2000,True,3
4,bike_lane_no_parking,1.5,0.0,35,1,2,500,False,2
5,bike_lane_no_parking,1.2,0.0,60,2,2,3000,True,2
6,bike_lane_with_parking,1.8,2.6,40,2,2,4000,True,3
7,bike_lane_with_parking,1.5,2.5,55,3,2,5000,True,3
8,separated_path,2.0,0.0,25,1,1,200,False,1
9,mixed_traffic,NaN,0.0,45,1,1,800,False,2
